### Create a Delta Table

In [0]:
from delta.tables import *

DeltaTable.create(spark)\
  .tableName("delta_internal_demo")\
    .addColumn("Emp_id", "INT")\
      .addColumn("Emp_name", "STRING")\
        .addColumn("Gender", "STRING")\
          .addColumn("Salary", "INT")\
            .addColumn("Dept", "STRING")\
              .property("Description", "Delta Internal Architecture Demo")\
                .location("/FileStore/tables/delta/arch_demo")\
                  .execute()


Out[3]: <delta.tables.DeltaTable at 0x7fb2500e9b50>

In [0]:

%fs
ls /FileStore/tables/delta/arch_demo/_delta_log

path,name,size,modificationTime
dbfs:/FileStore/tables/delta/arch_demo/_delta_log/.s3-optimization-0,.s3-optimization-0,0,1762525185000
dbfs:/FileStore/tables/delta/arch_demo/_delta_log/.s3-optimization-1,.s3-optimization-1,0,1762525185000
dbfs:/FileStore/tables/delta/arch_demo/_delta_log/.s3-optimization-2,.s3-optimization-2,0,1762525185000
dbfs:/FileStore/tables/delta/arch_demo/_delta_log/00000000000000000000.crc,00000000000000000000.crc,2235,1762525202000
dbfs:/FileStore/tables/delta/arch_demo/_delta_log/00000000000000000000.json,00000000000000000000.json,1259,1762525185000


In [0]:
%fs

head /FileStore/tables/delta/arch_demo/_delta_log/00000000000000000000.json

{"commitInfo":{"timestamp":1762525184458,"userId":"6333491925723577","userName":"srishtishetty53@gmail.com","operation":"CREATE TABLE","operationParameters":{"isManaged":"false","description":null,"partitionBy":"[]","properties":"{\"Description\":\"Delta Internal Architecture Demo\"}"},"notebook":{"notebookId":"1472541531204446"},"clusterId":"1107-141305-674k90tc","isolationLevel":"WriteSerializable","isBlindAppend":true,"operationMetrics":{},"engineInfo":"Databricks-Runtime/12.2.x-scala2.12","txnId":"b9e53f9f-44c4-412a-8cb7-af1b36b65108"}}
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"b00a81ae-cf23-4ea8-af7e-47d65096433d","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"Emp_id\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"Emp_name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"Gender\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"Salary\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"Dept\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{"Description":"Delta Internal Architecture Demo"},"createdTime":1762525182923}}

In [0]:
%sql

SELECT * FROM delta_internal_demo

Emp_id,Emp_name,Gender,Salary,Dept


In [0]:
%sql
INSERT INTO delta_internal_demo VALUES(100, "Stephen", "M", 2000, "IT")


num_affected_rows,num_inserted_rows
1,1


In [0]:
%fs

head /FileStore/tables/delta/arch_demo/_delta_log/00000000000000000001.json

{"commitInfo":{"timestamp":1762525974818,"userId":"6333491925723577","userName":"srishtishetty53@gmail.com","operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"notebook":{"notebookId":"1472541531204446"},"clusterId":"1107-141305-674k90tc","readVersion":0,"isolationLevel":"WriteSerializable","isBlindAppend":true,"operationMetrics":{"numFiles":"1","numOutputRows":"1","numOutputBytes":"1521"},"engineInfo":"Databricks-Runtime/12.2.x-scala2.12","txnId":"359edc57-de5d-4641-a9c4-ea8c018f9c66"}}
{"add":{"path":"part-00000-d79a6c6a-5c1a-42f0-9721-580106550e08-c000.snappy.parquet","partitionValues":{},"size":1521,"modificationTime":1762525975000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"Emp_id\":100,\"Emp_name\":\"Stephen\",\"Gender\":\"M\",\"Salary\":2000,\"Dept\":\"IT\"},\"maxValues\":{\"Emp_id\":100,\"Emp_name\":\"Stephen\",\"Gender\":\"M\",\"Salary\":2000,\"Dept\":\"IT\"},\"nullCount\":{\"Emp_id\":0,\"Emp_name\":0,\"Gender\":0,\"Salary\":0,\"Dept\":0}}","tags":{"INSERTION_TIME":"1762525975000000","MIN_INSERTION_TIME":"1762525975000000","MAX_INSERTION_TIME":"1762525975000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}

### Trying to insert duplicate records

In [0]:
%sql
INSERT INTO delta_internal_demo VALUES
(101, 'Emily', 'F', 2500, 'HR'),
(102, 'Michael', 'M', 2200, 'Finance');

INSERT INTO delta_internal_demo VALUES
(101, 'Emily', 'F', 2500, 'HR'),
(102, 'Michael', 'M', 2200, 'Finance');

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
SELECT * FROM delta_internal_demo

Emp_id,Emp_name,Gender,Salary,Dept
101,Emily,F,2500,HR
102,Michael,M,2200,Finance
101,Emily,F,2500,HR
102,Michael,M,2200,Finance
100,Stephen,M,2000,IT


In [0]:
%sql
DELETE FROM delta_internal_demo WHERE Emp_id IN (102)

num_affected_rows
2


In [0]:
%sql
SELECT * FROM delta_internal_demo

Emp_id,Emp_name,Gender,Salary,Dept
100,Stephen,M,2000,IT
101,Emily,F,2500,HR
101,Emily,F,2500,HR


In [0]:
%fs

head /FileStore/tables/delta/arch_demo/_delta_log/00000000000000000010.json

{"commitInfo":{"timestamp":1762527474130,"userId":"6333491925723577","userName":"srishtishetty53@gmail.com","operation":"DELETE","operationParameters":{"predicate":"[\"(Emp_id#7005 = 102)\"]"},"notebook":{"notebookId":"1472541531204446"},"clusterId":"1107-141305-674k90tc","readVersion":9,"isolationLevel":"WriteSerializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"2","numRemovedBytes":"3054","numCopiedRows":"2","numDeletionVectorsAdded":"0","numDeletionVectorsRemoved":"0","numAddedChangeFiles":"0","executionTimeMs":"2363","numDeletedRows":"2","scanTimeMs":"1140","numAddedFiles":"2","numAddedBytes":"3014","rewriteTimeMs":"1223"},"engineInfo":"Databricks-Runtime/12.2.x-scala2.12","txnId":"03218719-4aeb-48f9-a8b3-43e3b5272268"}}
{"remove":{"path":"part-00000-5c1d39b5-9d0b-4bd1-986b-860db3fd5288-c000.snappy.parquet","deletionTimestamp":1762527474119,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":1527,"tags":{"INSERTION_TIME":"1762527443000000","MIN_INSERTION_TIME":"1762527443000000","MAX_INSERTION_TIME":"1762527443000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}
{"remove":{"path":"part-00000-5d565172-eefc-4c61-a0a4-2b86e0c3ac18-c000.snappy.parquet","deletionTimestamp":1762527474119,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":1527,"tags":{"INSERTION_TIME":"1762527439000000","MIN_INSERTION_TIME":"1762527439000000","MAX_INSERTION_TIME":"1762527439000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}
{"add":{"path":"part-00000-93208a6f-2ac6-48aa-97c5-cfa7106a7ed2-c000.snappy.parquet","partitionValues":{},"size":1507,"modificationTime":1762527474000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"Emp_id\":101,\"Emp_name\":\"Emily\",\"Gender\":\"F\",\"Salary\":2500,\"Dept\":\"HR\"},\"maxValues\":{\"Emp_id\":101,\"Emp_name\":\"Emily\",\"Gender\":\"F\",\"Salary\":2500,\"Dept\":\"HR\"},\"nullCount\":{\"Emp_id\":0,\"Emp_name\":0,\"Gender\":0,\"Salary\":0,\"Dept\":0}}","tags":{"MAX_INSERTION_TIME":"1762527439000000","INSERTION_TIME":"1762527439000000","MIN_INSERTION_TIME":"1762527439000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}
{"add":{"path":"part-00001-789f019c-4f2c-4454-bc7c-e05206e4c615-c000.snappy.parquet","partitionValues":{},"size":1507,"modificationTime":1762527474000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"Emp_id\":101,\"Emp_name\":\"Emily\",\"Gender\":\"F\",\"Salary\":2500,\"Dept\":\"HR\"},\"maxValues\":{\"Emp_id\":101,\"Emp_name\":\"Emily\",\"Gender\":\"F\",\"Salary\":2500,\"Dept\":\"HR\"},\"nullCount\":{\"Emp_id\":0,\"Emp_name\":0,\"Gender\":0,\"Salary\":0,\"Dept\":0}}","tags":{"MAX_INSERTION_TIME":"1762527443000000","INSERTION_TIME":"1762527443000000","MIN_INSERTION_TIME":"1762527443000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}

### Reading the checkpointed delta log parquet file

In [0]:
spark.read.parquet("/FileStore/tables/delta/arch_demo/_delta_log/00000000000000000010.checkpoint.parquet").display()

txn,add,remove,metaData,protocol,rowIdHighWaterMark
null,null,null,null,"List(1, 2, null, null)",null
null,null,null,"List(b00a81ae-cf23-4ea8-af7e-47d65096433d, null, null, List(parquet, Map()), {""type"":""struct"",""fields"":[{""name"":""Emp_id"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Emp_name"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Gender"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Salary"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Dept"",""type"":""string"",""nullable"":true,""metadata"":{}}]}, List(), Map(Description -> Delta Internal Architecture Demo), 1762525182923)",null,null
null,null,"List(part-00000-2c81753f-dd6a-47f4-9efb-35077a3f730b-c000.snappy.parquet, 1762526698443, false, true, Map(), 1527, null, null)",null,null,null
null,null,"List(part-00000-4151f1ec-4a50-49b9-ba6a-17e002b43aab-c000.snappy.parquet, 1762526698443, false, true, Map(), 1527, null, null)",null,null,null
null,null,"List(part-00000-5c1d39b5-9d0b-4bd1-986b-860db3fd5288-c000.snappy.parquet, 1762527474119, false, true, Map(), 1527, null, null)",null,null,null
null,null,"List(part-00000-5d565172-eefc-4c61-a0a4-2b86e0c3ac18-c000.snappy.parquet, 1762527474119, false, true, Map(), 1527, null, null)",null,null,null
null,null,"List(part-00000-823f03ac-649f-4517-bc79-96763e09ed14-c000.snappy.parquet, 1762526698443, false, true, Map(), 1527, null, null)",null,null,null
null,null,"List(part-00000-9222b0da-753c-48ca-826d-4c851058cc29-c000.snappy.parquet, 1762526698443, false, true, Map(), 1527, null, null)",null,null,null
null,"List(part-00000-93208a6f-2ac6-48aa-97c5-cfa7106a7ed2-c000.snappy.parquet, Map(), 1507, 1762527474000, false, Map(MAX_INSERTION_TIME -> 1762527439000000, INSERTION_TIME -> 1762527439000000, MIN_INSERTION_TIME -> 1762527439000000, OPTIMIZE_TARGET_SIZE -> 268435456), null, null, {""numRecords"":1,""minValues"":{""Emp_id"":101,""Emp_name"":""Emily"",""Gender"":""F"",""Salary"":2500,""Dept"":""HR""},""maxValues"":{""Emp_id"":101,""Emp_name"":""Emily"",""Gender"":""F"",""Salary"":2500,""Dept"":""HR""},""nullCount"":{""Emp_id"":0,""Emp_name"":0,""Gender"":0,""Salary"":0,""Dept"":0}}, List(1, List(101, Emily, F, 2500, HR), List(101, Emily, F, 2500, HR), List(0, 0, 0, 0, 0)))",null,null,null,null
null,null,"List(part-00000-cacfd394-7615-409b-9533-59f53986d073-c000.snappy.parquet, 1762526698443, false, true, Map(), 1527, null, null)",null,null,null
